In [3]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import BaseScheduler
from mesa.space import MultiGrid
import numpy as np
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule, TextElement
from mesa.datacollection import DataCollector
import random
import math
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# this agent is a seat          
class seat_agent(Agent):
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.occupied=0  

    def step(self):
        #print(self.dist_calc)
        return

In [63]:
# this agent is the passenger           
class passenger(Agent):
    def __init__(self, pos, model, seat):
        super().__init__(pos, model)
        self.state=1 #1 is walking, 0 stowing bags, 3 is sitting in seat
        self.seat_number = seat
        self.luggage = random.randint(1,2)
        
  
  #function for movement along the plane    
    def move(self):
        neighbors = []
        x, y = self.pos
        for dx in [1]:
            for dy in [0]:
                neighbors.append((x+dx, y+dy))
        possible_steps = neighbors
        if len(self.model.grid.get_cell_list_contents([possible_steps[0]]))==0:
                new_position = self.random.choice(possible_steps)
                self.model.grid.move_agent(self, new_position)
   
   
    #Check if next to a seat   
    def check_seat(self,cell):
            for i in self.model.grid.get_cell_list_contents([cell]):
                if type(i) is seat_agent:
                    return i

   #function to check if arrived at a destination by assigned seat to adjacent seat    
    def check_arrival(self):       
        loc = []
        x, y = self.pos
        for dx in [0]:
            for dy in [1]:
                loc.append((x+dx, y+dy))       
        location=self.check_seat(loc[0])
        if seat_to_grid[self.seat_number][0]==x: 
            print("A3333333")
            self.timer=0
            self.state=3
            #self.sit(seat_to_grid[self.seat_number]) 

    # Sit on assigned seat    
    def sit(self,seat):
                     
        if self.state==3:
            # seat value taken from args
            self.check_seat(seat_to_grid[self.seat_number]).occupied=1
            self.model.grid.move_agent(self, seat_to_grid[self.seat_number])
            self.state=0            

    def step(self):
        
        if self.state==1:
            self.check_arrival()
            if self.state==1:
                self.move()
                print(self.seat_number,self.pos)
                self.check_arrival()
        elif self.state==3:
            if self.timer<10: #how long it takes to stow luggage
                
                self.timer+=1
                print("my timer is ", self.timer)
            else:
                print("i need to sit now")
                self.sit(seat_to_grid[self.seat_number])
                
                
        return

In [139]:
#Model


class plane(Model):
    """A model with some number of agents."""
    
    def __init__(self,mode=1,width=30, height=7, N=180):
        super().__init__()
        
        #### random rows
        plane_rows=[*range(1,31)]
        plane_cols=['a','b','c','d','e','f']
        plane_seats=[]
        for row in plane_rows:
            for col in plane_cols:
                plane_seats.append((row,col))
        plane_seats
        actual_rows=[*range(0,30)]
        actual_cols=[*range(0,7)]
        actual_grid=[]
        for row in actual_rows:
            for col in actual_cols:
                actual_grid.append((row,col))

        actual_grid_seats=[]
        for i in actual_grid:
            if i[1]!=3:
                actual_grid_seats.append(i)
        seat_to_grid= dict(zip(plane_seats, actual_grid_seats))
        
        ####BACK TO FRONT LOADING
        
        
        def flatten(plane_seats_sector):   
            flat_list = []
            for sublist in plane_seats_sector:
                for item in sublist:
                    flat_list.append(item)
            return flat_list   

        all_sectors=[]
        plane_seats_sector=[]
        p=0
        v=30
        for i in range(6):
            plane_seats_sector=[]
            plane_seats_sector.append(plane_seats[p:v])
            flp=flatten(plane_seats_sector)
            random.shuffle(flp)

            all_sectors.append(flp)
            p+=30
            v+=30
        all_sectors
        btf=flatten(all_sectors)
        
        
        ### Model Params
        
        self.mode=mode
        self.width=width
        self.height=height
        self.grid = MultiGrid(width, height, False)
        self.place=0
        self.schedule = BaseScheduler(self)
        self.all_passengers=[]
        
        if self.mode==1:
            
            self.tickets=plane_seats
        elif self.mode==2:
            self.tickets=btf
        
        self.builder=[]
        type_distribution = np.genfromtxt("plane.txt")
        self.counter=0.01
        
        
        #create seat
        for _,x, y in self.grid.coord_iter():
            if type_distribution[x,y]==1: #1 is the chosen identifier for a seat in the txt doc
                new_seat=seat_agent((x,y),self)
                self.grid.place_agent(new_seat,(x, y))
                self.schedule.add(new_seat)

        #create passengers 
        
        for i in range(180):      
            

            if True:
                a = passenger(i, self, self.tickets[-1])
                self.tickets.pop()
                self.all_passengers.append(a)
                
                if self.mode==1:
                    random.shuffle(self.all_passengers)
                    
                    
                
                
            
        
            
        


         #data
        """self.datacollector = DataCollector(
            model_reporters={"Parcels Delivered by Bikes": comp_bike_parcels_del,
                             "Parcels Delivered by Vans": comp_van_parcels_del, "Average Movement Speed in Km/h":average_speed_cal,"Model-time Average Speed in Km/h":average_speed_cal_total}
            )"""
                
        self.running = True
        #self.datacollector.collect(self)

       
        
    def step(self):
        self.counter+=1
        
        
        self.schedule.step()
        
        if len(self.all_passengers)>0 and len(self.grid.get_cell_list_contents([(0,3)]))==0:#[*range(0,2000)][self.place]%2==0:
                
                self.grid.place_agent(self.all_passengers[0],(0,3))
                self.schedule.add(self.all_passengers[0])
                self.all_passengers.pop(0)
        
        #self.datacollector.collect(self)
        #self.schedule.step()
        self.place+=1
        
            
          
            

In [80]:
def agent_portrayal(agent):
    if agent is None:
        return

    portrayal = {}

    if type(agent) is passenger:
        portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5, "Color" : "#32a883", "Layer":5,}

      
    elif type(agent) is seat_agent:
        
        portrayal = {"Shape": "rect",
                 "Filled": "true",
                "Layer": 3 , "w":1 , "h":1,}
        
        if agent.occupied== 0:
            portrayal["Shape"] = "seat_a.png"
        else:
            portrayal["Shape"] = "seat_o.png" 
            
            

    return portrayal



grid = CanvasGrid(agent_portrayal, 30, 7, 690,161)


In [140]:
port_num=9161



server = ModularServer(plane,
                       [grid], 
                       "Plane Model")

server.port = port_num
server.launch()

Interface starting at http://127.0.0.1:9161


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
(19, 'd') (1, 3)
{"type":"get_step","step":3}
(19, 'd') (2, 3)
(15, 'f') (1, 3)
{"type":"get_step","step":4}
(19, 'd') (3, 3)
(15, 'f') (2, 3)
(26, 'f') (1, 3)
{"type":"get_step","step":5}
(19, 'd') (4, 3)
(15, 'f') (3, 3)
(26, 'f') (2, 3)
(19, 'a') (1, 3)
{"type":"get_step","step":6}
(19, 'd') (5, 3)
(15, 'f') (4, 3)
(26, 'f') (3, 3)
(19, 'a') (2, 3)
(28, 'd') (1, 3)
{"type":"get_step","step":7}
(19, 'd') (6, 3)
(15, 'f') (5, 3)
(26, 'f') (4, 3)
(19, 'a') (3, 3)
(28, 'd') (2, 3)
(10, 'b') (1, 3)
{"type":"get_step","step":8}
(19, 'd') (7, 3)
(15, 'f') (6, 3)
(26, 'f') (5, 3)
(19, 'a') (4, 3)
(28, 'd') (3, 3)
(10, 'b') (2, 3)
(17, 'f') (1, 3)
{"type":"get_step","step":9}
(19, 'd') (8, 3)
(15, 'f') (7, 3)
(26, 'f') (6, 3)
(19, 'a') (5, 3)
(28, 'd') (4, 3)
(10, 'b') (3, 3)
(17, 'f') (2, 3)
(30, 'e') (1, 3)
{"type":"get_step","step":10}
(19, 'd') (9, 3)
(15, 'f') (8, 3)
(26, 'f') (7, 3)
(19, 'a') (6, 

{"type":"get_step","step":40}
my timer is  2
my timer is  8
(28, 'd') (17, 3)
my timer is  7
(30, 'e') (15, 3)
my timer is  10
i need to sit now
(23, 'd') (9, 3)
(26, 'e') (8, 3)
(28, 'e') (7, 3)
(10, 'd') (6, 3)
i need to sit now
(13, 'b') (3, 3)
(30, 'a') (2, 3)
(24, 'd') (1, 3)
{"type":"get_step","step":41}
my timer is  3
my timer is  9
(28, 'd') (17, 3)
my timer is  8
(30, 'e') (15, 3)
i need to sit now
(23, 'd') (10, 3)
(26, 'e') (9, 3)
(28, 'e') (8, 3)
(10, 'd') (7, 3)
(13, 'b') (4, 3)
(30, 'a') (3, 3)
(24, 'd') (2, 3)
(30, 'b') (1, 3)
{"type":"get_step","step":42}
my timer is  4
my timer is  10
(28, 'd') (17, 3)
my timer is  9
(30, 'e') (15, 3)
(23, 'd') (11, 3)
(26, 'e') (10, 3)
(28, 'e') (9, 3)
(10, 'd') (8, 3)
(13, 'b') (5, 3)
(30, 'a') (4, 3)
(24, 'd') (3, 3)
(30, 'b') (2, 3)
(11, 'd') (1, 3)
{"type":"get_step","step":43}
my timer is  5
i need to sit now
(28, 'd') (18, 3)
my timer is  10
(30, 'e') (15, 3)
(23, 'd') (12, 3)
(26, 'e') (11, 3)
(28, 'e') (10, 3)
(10, 'd') (9, 3)

{"type":"get_step","step":70}
my timer is  5
my timer is  3
(28, 'e') (24, 3)
(30, 'a') (14, 3)
(24, 'd') (13, 3)
(30, 'b') (12, 3)
my timer is  1
(27, 'e') (9, 3)
my timer is  2
(19, 'b') (6, 3)
(8, 'c') (5, 3)
(24, 'e') (4, 3)
(27, 'f') (3, 3)
(10, 'e') (2, 3)
(28, 'c') (1, 3)
(14, 'd') (0, 3)
{"type":"get_step","step":71}
my timer is  6
my timer is  4
(28, 'e') (24, 3)
(30, 'a') (15, 3)
(24, 'd') (14, 3)
(30, 'b') (13, 3)
my timer is  2
(27, 'e') (9, 3)
my timer is  3
(19, 'b') (6, 3)
(8, 'c') (5, 3)
(24, 'e') (4, 3)
(27, 'f') (3, 3)
(10, 'e') (2, 3)
(28, 'c') (1, 3)
(14, 'd') (0, 3)
{"type":"get_step","step":72}
my timer is  7
my timer is  5
(28, 'e') (24, 3)
(30, 'a') (16, 3)
(24, 'd') (15, 3)
(30, 'b') (14, 3)
my timer is  3
(27, 'e') (9, 3)
my timer is  4
(19, 'b') (6, 3)
(8, 'c') (5, 3)
(24, 'e') (4, 3)
(27, 'f') (3, 3)
(10, 'e') (2, 3)
(28, 'c') (1, 3)
(14, 'd') (0, 3)
{"type":"get_step","step":73}
my timer is  8
my timer is  6
(28, 'e') (24, 3)
(30, 'a') (17, 3)
(24, 'd') (16

...

{"type":"get_step","step":674}
my timer is  7
my timer is  5
my timer is  5
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') (1, 3)
(9, 'b') (0, 3)
{"type":"get_step","step":675}
my timer is  8
my timer is  6
my timer is  6
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') (1, 3)
(9, 'b') (0, 3)
{"type":"get_step","step":676}
my timer is  9
my timer is  7
my timer is  7
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') (1, 3)
(9, 'b') (0, 3)
{"type":"get_step","step":677}
my timer is  10
my timer is  8
my timer is  8
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') (1, 3)
(9, 'b') (0, 3)
{"type":"get_step","step":678}
i need to sit now
my timer is  9
my timer is  9
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') (1, 3)
(9, 'b') (0, 3)
{"type":"get_step","step":679}
my timer is  10
my timer is  10
(7, 'c') (5, 3)
(7, 'e') (4, 3)
(9, 'a') (3, 3)
(7, 'f') (2, 3)
(8, 'b') 

{"type":"get_step","step":728}
my timer is  1
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)
(6, 'e') (2, 3)
(8, 'a') (1, 3)
(10, 'd') (0, 3)
{"type":"get_step","step":729}
my timer is  2
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)
(6, 'e') (2, 3)
(8, 'a') (1, 3)
(10, 'd') (0, 3)
{"type":"get_step","step":730}
my timer is  3
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)
(6, 'e') (2, 3)
(8, 'a') (1, 3)
(10, 'd') (0, 3)
{"type":"get_step","step":731}
my timer is  4
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)
(6, 'e') (2, 3)
(8, 'a') (1, 3)
(10, 'd') (0, 3)
{"type":"get_step","step":732}
my timer is  5
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)
(6, 'e') (2, 3)
(8, 'a') (1, 3)
(10, 'd') (0, 3)
{"type":"get_step","step":733}
my timer is  6
(9, 'f') (7, 3)
(8, 'c') (6, 3)
(9, 'e') (5, 3)
(6, 'b') (4, 3)
(7, 'b') (3, 3)

{"type":"get_step","step":779}
my timer is  3
my timer is  2
(10, 'b') (7, 3)
(8, 'e') (6, 3)
my timer is  2
(10, 'c') (4, 3)
(9, 'd') (3, 3)
(10, 'f') (2, 3)
(7, 'd') (1, 3)
(2, 'b') (0, 3)
{"type":"get_step","step":780}
my timer is  4
my timer is  3
(10, 'b') (7, 3)
(8, 'e') (6, 3)
my timer is  3
(10, 'c') (4, 3)
(9, 'd') (3, 3)
(10, 'f') (2, 3)
(7, 'd') (1, 3)
(2, 'b') (0, 3)
{"type":"get_step","step":781}
my timer is  5
my timer is  4
(10, 'b') (7, 3)
(8, 'e') (6, 3)
my timer is  4
(10, 'c') (4, 3)
(9, 'd') (3, 3)
(10, 'f') (2, 3)
(7, 'd') (1, 3)
(2, 'b') (0, 3)
{"type":"get_step","step":782}
my timer is  6
my timer is  5
(10, 'b') (7, 3)
(8, 'e') (6, 3)
my timer is  5
(10, 'c') (4, 3)
(9, 'd') (3, 3)
(10, 'f') (2, 3)
(7, 'd') (1, 3)
(2, 'b') (0, 3)
{"type":"get_step","step":783}
my timer is  7
my timer is  6
(10, 'b') (7, 3)
(8, 'e') (6, 3)
my timer is  6
(10, 'c') (4, 3)
(9, 'd') (3, 3)
(10, 'f') (2, 3)
(7, 'd') (1, 3)
(2, 'b') (0, 3)
{"type":"get_step","step":784}
my timer is  8

{"type":"get_step","step":842}
my timer is  7
my timer is  7
my timer is  6
{"type":"get_step","step":843}
my timer is  8
my timer is  8
my timer is  7
{"type":"get_step","step":844}
my timer is  9
my timer is  9
my timer is  8
{"type":"get_step","step":845}
my timer is  10
my timer is  10
my timer is  9
{"type":"get_step","step":846}
i need to sit now
i need to sit now
my timer is  10
{"type":"get_step","step":847}
i need to sit now
{"type":"get_step","step":848}
(2, 'e') (1, 3)
A3333333
{"type":"get_step","step":849}
my timer is  1
(4, 'a') (0, 3)
{"type":"get_step","step":850}
my timer is  2
(4, 'a') (0, 3)
{"type":"get_step","step":851}
my timer is  3
(4, 'a') (0, 3)
{"type":"get_step","step":852}
my timer is  4
(4, 'a') (0, 3)
{"type":"get_step","step":853}
my timer is  5
(4, 'a') (0, 3)
{"type":"get_step","step":854}
my timer is  6
(4, 'a') (0, 3)
{"type":"get_step","step":855}
my timer is  7
(4, 'a') (0, 3)
{"type":"get_step","step":856}
my timer is  8
(4, 'a') (0, 3)
{"type":"g

{"type":"get_step","step":956}
my timer is  7
(5, 'a') (3, 3)
(4, 'd') (2, 3)
(5, 'f') (1, 3)
(4, 'e') (0, 3)
{"type":"get_step","step":957}
my timer is  8
(5, 'a') (3, 3)
(4, 'd') (2, 3)
(5, 'f') (1, 3)
(4, 'e') (0, 3)
{"type":"get_step","step":958}
my timer is  9
(5, 'a') (3, 3)
(4, 'd') (2, 3)
(5, 'f') (1, 3)
(4, 'e') (0, 3)
{"type":"get_step","step":959}
my timer is  10
(5, 'a') (3, 3)
(4, 'd') (2, 3)
(5, 'f') (1, 3)
(4, 'e') (0, 3)
{"type":"get_step","step":960}
i need to sit now
(5, 'a') (4, 3)
A3333333
(4, 'd') (3, 3)
A3333333
(5, 'f') (2, 3)
(4, 'e') (1, 3)
{"type":"get_step","step":961}
my timer is  1
my timer is  1
(5, 'f') (2, 3)
(4, 'e') (1, 3)
(5, 'b') (0, 3)
{"type":"get_step","step":962}
my timer is  2
my timer is  2
(5, 'f') (2, 3)
(4, 'e') (1, 3)
(5, 'b') (0, 3)
{"type":"get_step","step":963}
my timer is  3
my timer is  3
(5, 'f') (2, 3)
(4, 'e') (1, 3)
(5, 'b') (0, 3)
{"type":"get_step","step":964}
my timer is  4
my timer is  4
(5, 'f') (2, 3)
(4, 'e') (1, 3)
(5, 'b')

[7, 6, 5, 4, 23, 1]